In [1]:
import pandas as pd
import numpy as np
from chromadb import PersistentClient
from typing import List
import uuid
import ast

# from npl.merge_market_dataframes import vivanda

# from npl.npl import batch, batch_size

In [2]:
plazavea = pd.read_parquet('plazavea_con_embeddings.parquet')
wong = pd.read_parquet('wong_con_embeddings.parquet')
metro = pd.read_parquet('metro_con_embeddings.parquet')
vivanda = pd.read_parquet('vivanda_con_embeddings.parquet')

In [3]:
data = pd.concat([plazavea, wong, metro, vivanda], ignore_index=True)

In [4]:
# Cargar datos
# data = pd.read_parquet('productos_con_embeddings.parquet')
print(f"Datos cargados: {len(data)} registros")

Datos cargados: 102107 registros


In [5]:
data['categories_txt'] = data['categories'].apply(lambda x: ', '.join(ast.literal_eval(x) if isinstance(x, str) else x))

In [6]:
# Inicializar ChromaDB
client = PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="productos")

In [7]:
# df = data.iloc[:5000] # Para pruebas con pocas filas
df = data
len(df)

102107

In [16]:
df.sample(n=5)

,product_name,link,descripcion_producto,price,source,categories,embedding,categories_txt
95394,Televisor LG OLED 48'' UHD 4K ThinQ AI OLED48A2,https://www.vivanda.com.pe/televisor-lg-oled-4...,Vivanda,0.00,Vivanda,"[Tecnología, Televisores, TV OLED]","[-0.012823214754462242, -0.015416167676448822,...","Tecnología, Televisores, TV OLED"
3909,Combo Hogar Dispensador Lavavajilla Limpiavidr...,https://www.plazavea.com.pe/combo-hogar-dispen...,plazaVea,41.99,plazaVea,"[Limpieza, Accesorios de Limpieza, Otros Acces...","[-0.010566323064267635, 0.00403645820915699, 0...","Limpieza, Accesorios de Limpieza, Otros Acceso..."
34405,Seccional derecho Pitágora Celeste,https://www.plazavea.com.pe/seccional-derecho-...,plazaVea,6399.00,plazaVea,"[Muebles, Sofás y Sillones, Seccionales, O, se...","[0.006712367758154869, 0.017337163910269737, 0...","Muebles, Sofás y Sillones, Seccionales, O, sec..."
83259,Set Rosen Ferrara: Cabecera 1.5 Plazas + Velad...,https://www.metro.pe/set-rosen-ferrara-cabecer...,Metro,649.00,Metro,"[Dormitorio, Muebles de Dormitorio, Cabeceras]","[-0.008545102551579475, -0.001420014537870884,...","Dormitorio, Muebles de Dormitorio, Cabeceras"
101840,Vino SEÑORÍO DE NAJAR Borgoña Botella 750ml,https://www.vivanda.com.pe/vino-senorio-de-naj...,Vivanda,18.50,Vivanda,"[Vinos, licores y cervezas, Vino Tinto]","[-0.01823776215314865, -0.031968794763088226, ...","Vinos, licores y cervezas, Vino Tinto"


In [19]:
embeddings = []
metadatas = []
documents = []
for _, row in df.iterrows():
    emb = row['embedding']
    embeddings.append(emb)

    metadata = {
            'producto': str(row['product_name']),
            'descripcion': str(row['descripcion_producto']),
            'precio': float(row['price']),
            'market': str(row['source']),
            'categorias': str(row['categories_txt']),
            'url': str(row['link'])
        }
    metadatas.append(metadata)

    document = f"Producto: {row['product_name']}. Descripción: {row['descripcion_producto']}. Categorías: {row['categories_txt']}"
    documents.append(document)


KeyboardInterrupt: 

In [9]:
|ids = [str(uuid.uuid4()) for _ in range(len(df))]
total_records = len(ids)
batch_size = 200
for i in range(0, total_records, batch_size):
    end_idx = min(i + batch_size, total_records)

    batch_ids = ids[i:end_idx]
    batch_embeddings = embeddings[i:end_idx]
    batch_metadatas = metadatas[i:end_idx]
    batch_documents = documents[i:end_idx]

    try:
        collection.add(
            ids=batch_ids,
            embeddings=batch_embeddings,
            metadatas=batch_metadatas,
            documents=batch_documents
        )
        print(f"Insertados registros {i} a {end_idx} de {total_records}")
    except Exception as e:
        print(f"Error en lote {i}-{end_idx}: {str(e)}")
        if batch_embeddings:
            print(f"Dimensión del embedding: {len(batch_embeddings[0])}")
        continue

Insertados registros 0 a 200 de 102107
Insertados registros 200 a 400 de 102107
Insertados registros 400 a 600 de 102107
Insertados registros 600 a 800 de 102107
Insertados registros 800 a 1000 de 102107
Insertados registros 1000 a 1200 de 102107
Insertados registros 1200 a 1400 de 102107
Insertados registros 1400 a 1600 de 102107
Insertados registros 1600 a 1800 de 102107
Insertados registros 1800 a 2000 de 102107
Insertados registros 2000 a 2200 de 102107
Insertados registros 2200 a 2400 de 102107
Insertados registros 2400 a 2600 de 102107
Insertados registros 2600 a 2800 de 102107
Insertados registros 2800 a 3000 de 102107
Insertados registros 3000 a 3200 de 102107
Insertados registros 3200 a 3400 de 102107
Insertados registros 3400 a 3600 de 102107
Insertados registros 3600 a 3800 de 102107
Insertados registros 3800 a 4000 de 102107
Insertados registros 4000 a 4200 de 102107
Insertados registros 4200 a 4400 de 102107
Insertados registros 4400 a 4600 de 102107
Insertados registros 4

In [10]:
collection.count()

102107

In [14]:
collection.peek(limit = 9555)

{'ids': ['1751663d-357c-4904-97e5-c2427eaa6cff',
  '4625f9c9-dc22-46e7-b9e0-38e05f9e47d9',
  'e824b911-a00d-4830-ad23-12c8c5f0e109',
  'c1b9bbc3-e528-44bc-ba9a-aededd1746b3',
  'dd59acca-3021-4428-bd0e-ba373c9223df',
  '1847341b-71d7-4f52-b064-7883b9ec1765',
  'a7d287d7-4978-45a8-bcce-a8932134bc10',
  '361b6ef9-1442-422b-89d7-ab32cf6ea42b',
  '293b3b46-05dc-4ff3-8419-5e3f97fbe0e5',
  '0b0d7c73-1561-4c56-b898-f37760aa1160',
  '17fc92f9-3ed1-49fc-b13e-510b4dde76bb',
  '5b8b302c-6282-4f5e-9f2e-442290c922fb',
  'b3188538-15ee-4802-b306-48d8cb1ec642',
  'bf38785f-8d61-4c0b-8ac5-d1c5808c8853',
  '34dfbc47-9f6e-465b-8001-6ea67c8a8be8',
  '3e153356-dd63-4cf0-aa74-37dc583b017f',
  '90853b4f-2172-402d-9fc1-2e658b25e898',
  '08e0651c-b93c-4538-b186-732c081daa00',
  'a93fe67f-3f25-431d-bd85-558d76737b9f',
  'd09aae68-9c2c-4421-bce4-53ee6e70784e',
  'a13def8f-c325-4086-b36c-d6d12c1292fd',
  '5f81a528-548a-40ec-ab98-55b7dea35a54',
  '0b2e2118-278b-4e7d-b6a1-142127ff2fc6',
  'f9903a90-bb49-42b4-b1bb-